In [1]:
import sacrebleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-17 20:19:09.182511: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 20:19:09.208535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 20:19:09.703033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning

In [4]:
# Load the data
with open("user1.pkl", 'rb') as file1, open("output_emaps.pkl", 'rb') as file2:
    reference = pickle.load(file1)
    hypothesis = pickle.load(file2)

In [5]:
sacrebleu_score = sacrebleu.corpus_bleu(hypothesis, [reference])
sacrebleu_score = sacrebleu_score.score
sacrebleu_score

0.6506575526279764

In [9]:
from rouge_score import rouge_scorer

rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [rouge_scorer.score(ref, hyp)['rougeL'].fmeasure for ref, hyp in zip(reference, hypothesis)]
average_rouge_l_score = np.mean(rouge_l_scores)
print(f"Average ROUGE-L score: {average_rouge_l_score}")

Average ROUGE-L score: 0.0011655011655011655


In [10]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('snunlp/KR-SBERT-Medium-extended-klueNLItriplet_PARpair_QApair-klueSTS')
reference_embeddings = model.encode(reference, convert_to_tensor=True)
hypothesis_embeddings = model.encode(hypothesis, convert_to_tensor=True)
# Calculate cosine similarities
cosine_scores = util.cos_sim(reference_embeddings, hypothesis_embeddings)
avg_sts_score = np.mean([cosine_scores[i][i].item() for i in range(len(reference))])
print(f"STS: {avg_sts_score}")

STS: 0.37631496534566666


In [11]:
len(reference)

1716